In [16]:
import requests
import pandas as pd
import datetime

headers = {"User-Agent": "russ@sunriseanalysis.com"}

ticker = "AMD"
concept = "NetIncomeLoss"

In [17]:
def get_cik(ticker):
    """
    Get the CIK for a given ticker.
    Args:
        ticker (str): The ticker symbol of the company.
    Returns:
        str: The CIK for the company.
    """
    ticker = ticker.upper()
    headers = {"User-Agent": "russ@sunriseanalysis.com"}
    # Get the tickers JSON file
    tickers_json = requests.get(
        "https://www.sec.gov/files/company_tickers.json", headers=headers
    )
    # Create a DataFrame of the tickers
    cik_df = pd.DataFrame.from_dict(tickers_json.json(), orient="index")
    cik_df["cik_str"] = cik_df["cik_str"].astype(str).str.zfill(10)
    # Get the CIK for the given ticker
    cik = cik_df[cik_df["ticker"] == ticker]["cik_str"].values[0]
    return cik

In [18]:
cik = get_cik(ticker)

In [19]:
concept_url = (
    f"https://data.sec.gov/api/xbrl/companyconcept/CIK{cik}/us-gaap/{concept}.json"
)
concept_df = pd.DataFrame.from_dict(requests.get(concept_url, headers=headers).json())

In [20]:
concept_units = pd.DataFrame.from_dict(concept_df["units"]["USD"])
for col in ["start", "end", "filed"]:
    concept_units[col] = pd.to_datetime(concept_units[col])
df = concept_units
df["time_delta"] = df["end"] - df["start"]
df["val"] = df["val"] / 1_000_000
df.rename(columns={"val": "value(millions)"}, inplace=True)

annual_data = df[df["time_delta"] > pd.Timedelta(days=350)]
df = df.drop(annual_data.index)
annual_data = annual_data.drop_duplicates(subset="value(millions)", keep="last")
annual_data = annual_data.dropna(subset="frame")
annual_data = annual_data.drop(
    columns=["time_delta", "filed", "form", "fy", "fp"]
).reset_index(drop=True)

annual_data.rename(columns={"frame": "fiscal_year"}, inplace=True)
extra_data = df[df["time_delta"] > pd.Timedelta(days=115)]
df = df.drop(extra_data.index)
extra_data.reset_index(drop=True, inplace=True)
df = df.drop_duplicates(subset="value(millions)", keep="last")
df = df.drop(columns=["time_delta", "filed", "form", "fy", "fp", "accn"])
df = df.dropna(subset=["frame"])
df = df.reset_index(drop=True)

In [21]:
def map_to_quarter(end_date):
    month_diff = (end_date.month - end_month + 9) % 12
    if 0 <= month_diff < 3:
        return 1
    elif 3 <= month_diff < 6:
        return 2
    elif 6 <= month_diff < 9:
        return 3
    else:  # 9, 10, 11
        return 4


end_month = annual_data.iloc[0]["end"].month
df["quarter"] = df["end"].apply(map_to_quarter)

In [22]:
start_year = int(df["frame"].min()[2:6])  # extract start year from the minimum frame
end_year = int(df["frame"].max()[2:6])  # extract end year from the maximum frame
frames = [
    f"CY{year}Q{quarter}"
    for year in range(start_year, end_year + 1)
    for quarter in range(1, 5)
]
df_full = pd.DataFrame({"frame": frames})
# Step 2: Merge the original DataFrame with the new DataFrame
merged = pd.merge(df_full, df, how="left", on="frame")

# Step 3: Sort the DataFrame by frame
merged = merged.sort_values(by="frame").reset_index(drop=True)

In [23]:
merged["next_start"] = merged["start"].shift(-1)
merged["prev_end"] = merged["end"].shift()

# For missing quarters, set the 'start' date as the day after the end of the previous quarter,
# and the 'end' date as the day before the start of the next quarter
merged.loc[merged["start"].isna(), "start"] = merged["prev_end"] + pd.Timedelta(days=1)
merged.loc[merged["end"].isna(), "end"] = merged["next_start"] - pd.Timedelta(days=1)

# Drop the 'next_start' and 'prev_end' columns as they are no longer needed
merged.drop(["next_start", "prev_end"], axis=1, inplace=True)

In [24]:
today = datetime.datetime.today()
# Calculate the start date of the current quarter
if today.month < 4:
    quarter_start = datetime.datetime(today.year - 1, 12, 1)
elif today.month < 7:
    quarter_start = datetime.datetime(today.year, 3, 1)
elif today.month < 10:
    quarter_start = datetime.datetime(today.year, 6, 1)
else:
    quarter_start = datetime.datetime(today.year, 9, 1)
# Remove rows where the 'start' date is on or after the start of the current quarter
merged = merged[merged["start"] < quarter_start]

In [25]:
# Define a helper function to fill the quarter column
def fill_quarter(row, df):
    if pd.isnull(row["quarter"]):
        previous_index = row.name - 1
        next_index = row.name + 1 if row.name < len(df) - 1 else row.name

        if previous_index >= 0 and not pd.isnull(df.loc[previous_index, "quarter"]):
            previous_quarter = df.loc[previous_index, "quarter"]
            row["quarter"] = 1 if previous_quarter == 4 else previous_quarter + 1
        elif not pd.isnull(df.loc[next_index, "quarter"]):
            next_quarter = df.loc[next_index, "quarter"]
            row["quarter"] = 4 if next_quarter == 1 else next_quarter - 1
    return row


# Apply the function to each row in the DataFrame
merged = merged.apply(lambda row: fill_quarter(row, merged), axis=1)
merged["quarter"] = merged["quarter"].astype(int)

In [26]:
def find_fiscal_year(date, annual_data):
    for idx, row in annual_data.iterrows():
        if row["start"] <= date <= row["end"]:
            return row["fiscal_year"]

    # If the date is after the end of the last fiscal year in annual_data, return the next fiscal year
    last_fiscal_year_end = annual_data["end"].max()
    if date > last_fiscal_year_end:
        # Extract the last year, increment it by one, and return it as a fiscal year
        last_year = int(annual_data.iloc[-1]["fiscal_year"][2:])
        next_year = last_year + 1
        return "CY" + str(next_year)

    return None


# Apply the function to the 'end' column in 'df'
merged["fiscal_year"] = merged["end"].apply(find_fiscal_year, args=(annual_data,))

In [27]:
def fill_missing_quarters(df, annual_data):
    # Iterate over each row in the dataframe
    for idx, row in df.iterrows():
        # Check if the value for the quarter is missing
        if pd.isnull(row["value(millions)"]):
            # Check if there are three other quarters in the same fiscal year
            other_quarters = df[
                (df["fiscal_year"] == row["fiscal_year"]) & (df.index != idx)
            ]["value(millions)"]
            if other_quarters.count() == 3:
                # If there are, find the corresponding annual data
                annual_value = annual_data[
                    annual_data["fiscal_year"] == row["fiscal_year"]
                ]["value(millions)"].values[0]
                # Calculate the sum of the other three quarters
                sum_of_quarters = other_quarters.sum()
                # Fill the missing value with the difference
                df.at[idx, "value(millions)"] = annual_value - sum_of_quarters
    return df


# Apply the function to the DataFrame
merged = fill_missing_quarters(merged, annual_data)

In [28]:
merged

,frame,start,end,value(millions),quarter,fiscal_year
0,CY2011Q1,2010-12-26,2011-04-02,510.0,1,CY2011
1,CY2011Q2,2011-04-03,2011-07-02,61.0,2,CY2011
2,CY2011Q3,2011-07-03,2011-10-01,97.0,3,CY2011
3,CY2011Q4,2011-10-02,2011-12-31,-177.0,4,CY2011
4,CY2012Q1,2012-01-01,2012-03-31,-590.0,1,CY2012
5,CY2012Q2,2012-04-01,2012-06-30,37.0,2,CY2012
6,CY2012Q3,2012-07-01,2012-09-29,-157.0,3,CY2012
7,CY2012Q4,2012-09-30,2012-12-29,-473.0,4,CY2012
8,CY2013Q1,2012-12-30,2013-03-30,-146.0,1,CY2013
9,CY2013Q2,2013-03-31,2013-06-29,-74.0,2,CY2013


In [29]:
annual_data

,start,end,value(millions),accn,fiscal_year
0,2009-12-27,2010-12-25,471.0,0001193125-13-069422,CY2010
1,2010-12-26,2011-12-31,491.0,0001193125-14-057240,CY2011
2,2012-01-01,2012-12-29,-1183.0,0001193125-15-054362,CY2012
3,2012-12-30,2013-12-28,-83.0,0000002488-16-000111,CY2013
4,2013-12-29,2014-12-27,-403.0,0000002488-17-000043,CY2014
5,2014-12-28,2015-12-26,-660.0,0000002488-18-000042,CY2015
6,2015-12-27,2016-12-31,-498.0,0000002488-19-000011,CY2016
7,2017-01-01,2017-12-30,-33.0,0000002488-20-000008,CY2017
8,2017-12-31,2018-12-29,337.0,0001628280-21-001185,CY2018
9,2018-12-30,2019-12-28,341.0,0000002488-22-000016,CY2019
